<a href="https://colab.research.google.com/github/vipulSharma18/Automatic-Emotion-Recognition-on-DEAP-Dataset/blob/main/2_level_Ensemble_of_Light_Pyramidal_1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Relevant Libraries

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Softmax
from sklearn.metrics import classification_report
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# GPU Check

In [3]:
print(tf.version.VERSION)
print(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.4.1
[]
Num GPUs Available:  0


# Data Augmentation

## Load Data from .dat files into a np array of 1280 x 32 x 8064 size

In [4]:
all_sub_data = []
subjects_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
for sub in subjects_list:
    path = "/content/drive/MyDrive/major project/data_preprocessed_python/s"+sub+".dat"
    x = pickle.load(open(path, 'rb'), encoding = 'latin1')
    sub_data = x['data']
    sub_eeg = sub_data[:, :32, :]  #indexing EEG signals from physiological data
    all_sub_data.extend(sub_eeg)
all_sub_data = np.array(all_sub_data)

In [5]:
all_sub_data.shape

(1280, 32, 8064)

## Z-score normalization of each EEG signal, resultant np.array is all_sub_data

In [6]:
for sub in range(all_sub_data.shape[0]):
  all_sub_data[sub] = zscore(all_sub_data[sub], axis = 1)  #zscore normalize each channel

## Label Loading into np array of 1280 x 1 named, sub_labels

In [7]:
labels = pd.read_excel("/content/drive/MyDrive/major project/metadata/Labels.xls")
sub_labels = labels["Valence-Arousal Model Quadrant"].astype('int')
sub_labels

0       3
1       3
2       3
3       1
4       2
       ..
1275    1
1276    1
1277    1
1278    1
1279    2
Name: Valence-Arousal Model Quadrant, Length: 1280, dtype: int64

## One-Hot encoding of labels  
> sub_labels: (1280,4)

In [8]:
lb = preprocessing.LabelBinarizer()
sub_labels = lb.fit_transform(sub_labels)
print(lb.classes_)
print(sub_labels.shape)
print(sub_labels)

[0 1 2 3]
(1280, 4)
[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 ...
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]]


In [9]:
sub_labels.shape

(1280, 4)

## Generating Train Test Splits,  
> X_train, y_train: (1152,32,8064), (1152,4)  
> X_test, y_test: (128,32,8064), (128,4)  

In [10]:
X_train, X_test, y_train, y_test = train_test_split(all_sub_data, sub_labels, test_size = 0.1, random_state = 42,shuffle = True)

In [11]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1152, 32, 8064), (1152, 4), (128, 32, 8064), (128, 4))

## Repetition of Labels for Windowing of training data  

> y_train_12, y_train_6, y_train_4 of shapes(?,4): 13824, 6912 and 4608 respectively 


In [12]:
#12,6 and 4 subsignals are generated from 8064 length EEG signal, labels repeated accordingly
y_train_12 = np.repeat(y_train, 12, axis = 0)
y_train_6 = np.repeat(y_train, 6, axis = 0)
y_train_4 = np.repeat(y_train, 4, axis = 0)
print(y_train_12.shape, y_train_6.shape, y_train_4.shape)

(13824, 4) (6912, 4) (4608, 4)


## Windowing of EEG singals Channel wise

In [13]:
Channel_wise = np.transpose(X_train, (1,0,2))

In [14]:
print(Channel_wise[0].shape)

(1152, 8064)


In [15]:
def process_input(instances, sub_signals):
  #instances must be channel wise of shape (32, -1, 8064)
  samples = int(8064/sub_signals)
  transformed = []
  for i in range(instances.shape[0]):
    transformed.append(np.reshape(instances[i], (-1,672,1)))
  transformed = np.array(transformed)
  print(transformed.shape, 'is the shape obtained.')
  return transformed

In [17]:
channel_wise_12 = process_input(Channel_wise, 12)
print(channel_wise_12.shape)

(32, 13824, 672, 1) is the shape obtained.
(32, 13824, 672, 1)


In [ ]:
channel_wise_6 = process_input(Channel_wise, 6)
channel_wise_4 = process_input(Channel_wise, 4)

# Lightweight Pyramidal 1D CNN model

In [16]:
def create_models(dense_par=20, sub_signals=12):
  sample_size = int(8064/sub_signals)
  models = [0]*32
  for i in range(32):
    models[i] = Sequential()
    models[i].add(Conv1D(filters=32, kernel_size=5,strides = 3, input_shape=(sample_size, 1)))
    models[i].add(BatchNormalization())
    models[i].add(tf.keras.layers.Activation('relu'))
    models[i].add(Conv1D(filters=24, kernel_size=3,strides = 2))
    models[i].add(BatchNormalization())
    models[i].add(tf.keras.layers.Activation('relu'))
    models[i].add(Conv1D(filters=16, kernel_size=3,strides = 2))
    models[i].add(BatchNormalization())
    models[i].add(tf.keras.layers.Activation('relu'))
    models[i].add(Conv1D(filters=8, kernel_size=3,strides = 2))
    models[i].add(BatchNormalization())
    models[i].add(tf.keras.layers.Activation('relu'))
    models[i].add(Flatten())
    models[i].add(Dense(dense_par, activation='relu'))
    models[i].add(Dropout(rate = 0.5))
    models[i].add(Dense(4, activation = 'softmax'))
    models[i].compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001,beta_1=0.9,beta_2=0.999,epsilon=1e-08) , loss = tf.keras.losses.CategoricalCrossentropy(), metrics= 'accuracy')
  return models